```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNqNklFrwjAQgP9KyFMFba2MMfow0OmDIBYm-GKknM1pO9u0JKnixP--09oy3cv6EJK7y_fl0px5XEjkAd9mxTFOQFvGZp9CMfpMtdlpKBM2VWVl2QxOqOvMQjsfo0WH9XpM8MTa0gSeJ8GCRC1RufHGuCrzwjGFhmVaT7yXfv_tdT4Ze4LTzne2WDnGwi5Vu8664frO8LvSOMqKzR1PomiJ-ghxYqMhKAtZFB5QZ0gmVNGgP-hfBz-qFwM3NofG4LcKIdS20BiDsabRoZJ_OrWoc5QpWHxo-Iabrpz0V0F7ar_O-o_p_xnDyj5d7vRGC4kmMUYwKFmJmh0R9w1m6t9rWip7wgo-n7lfplCC15mQtrgU3kKwhZ4pU6VIyJlLnOV85jurxhbR31t3WjLv8px6glTSKzlfw4LbBHMUPKCpBL2_Oi5UV5X0BnAiU1toTprMYJdDZYvFScU8sLrCpmicAh00v1ddfgDI181Y)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNqNklFrwjAQgP9KyFMFba2MMfow0OmDIBYm-GKknM1pO9u0JKnixP--09oy3cv6EJK7y_fl0px5XEjkAd9mxTFOQFvGZp9CMfpMtdlpKBM2VWVl2QxOqOvMQjsfo0WH9XpM8MTa0gSeJ8GCRC1RufHGuCrzwjGFhmVaT7yXfv_tdT4Ze4LTzne2WDnGwi5Vu8664frO8LvSOMqKzR1PomiJ-ghxYqMhKAtZFB5QZ0gmVNGgP-hfBz-qFwM3NofG4LcKIdS20BiDsabRoZJ_OrWoc5QpWHxo-Iabrpz0V0F7ar_O-o_p_xnDyj5d7vRGC4kmMUYwKFmJmh0R9w1m6t9rWip7wgo-n7lfplCC15mQtrgU3kKwhZ4pU6VIyJlLnOV85jurxhbR31t3WjLv8px6glTSKzlfw4LbBHMUPKCpBL2_Oi5UV5X0BnAiU1toTprMYJdDZYvFScU8sLrCpmicAh00v1ddfgDI181Y)

Required steps:

1. **[CBS](https://dataderden.cbs.nl/ODataApi/OData/40086NED/TypedDataSet?$filter=(substringof(%272020W%27,%20Perioden)%20eq%20true)%20or%20(substringof(%272021W%27,%20Perioden)%20eq%20true)%20or%20(substringof(%272022W%27,%20Perioden)%20eq%20true)%20and%20(substringof(%27NL%27,%20RegioS)%20eq%20true)&$select=Perioden,%20RegioS,%20Overledenen_1)**: contains the `"registered deceased"` data collected by CBS that will be used for further processing.
2. **[AzureBlob]()**: contains the `"forecast of the deceased"` data collected by CBS that will be used for further processing.
3. **[Staging](#input-layer)**: inserts the raw `CBS` data within the `[VWSSTAGE].[CBS_DECEASED_PER_WEEK_NL]` table.
4. **[Staging Forecasts](#input-layer)**: inserts the raw `AzureBlob` data within the `[VWSSTAGE].[CBS_DECEASED_PER_WEEK_FORECAST_NL]` table.
5. **[Intermediate](#intermediate-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSINTER].[CBS_DECEASED_PER_WEEK_NL]` using business logic implemented in `[dbo].[SP_INSERT_IL_CBS_DECEASED_PER_WEEK_NL]`.
6. **[Intermediate Forecasts](#intermediate-layer)**: inserts the `Staging Forecasts` data and cast the correct `Database Types` within `[VWSINTER].[CBS_DECEASED_PER_WEEK_FORECAST_NL]` using business logic implemented in `[dbo].[SP_INSERT_IL_CBS_DECEASED_PER_WEEK_FORECAST_NL]`.
7. **[Deceased Per Week](#output-layer)**: processes the `Intermediate` and `Intermediate Forecasts` data to require the ***deceased per week***, and inserts the results within `[VWSDEST].[CBS_DECEASED_PER_WEEK_NL]` using business logic implemented in `[dbo].[SP_INSERT_OL_CBS_DECEASED_PER_WEEK_NL]`.
   - The `[VWSDEST].[V_CBS_DECEASED_PER_WEEK_NL]` is used to generate the json-objects `deceased_cbs`, within the `NL.json`.

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[CBS_DECEASED_PER_WEEK_NL]'))
CREATE TABLE [VWSSTAGE].[CBS_DECEASED_PER_WEEK_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[PERIODS] [VARCHAR](100),
	[DECEASED_1] [VARCHAR](100)
);
GO

### **<span style='color:cadetblue'>FORECAST</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[CBS_DECEASED_PER_WEEK_FORECAST_NL]'))
CREATE TABLE [VWSSTAGE].[CBS_DECEASED_PER_WEEK_FORECAST_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[PERIODS] [VARCHAR](100),
    [EXPECTED_AMOUNT_DECEASED] [VARCHAR](100),
    [LOWER_LIMIT] [VARCHAR](100),
    [UPPER_LIMIT] [VARCHAR](100)
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[CBS_DECEASED_PER_WEEK_NL]'))
CREATE TABLE [VWSINTER].[CBS_DECEASED_PER_WEEK_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[YEAR] [INT] NULL,
	[WEEK] [INT] NULL,
	[DECEASED_ACTUAL] [INT]
);
GO

### **<span style='color:cadetblue'>FORECAST</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[CBS_DECEASED_PER_WEEK_FORECAST_NL]'))
CREATE TABLE [VWSINTER].[CBS_DECEASED_PER_WEEK_FORECAST_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[YEAR] [INT] NULL,
	[WEEK] [INT] NULL,
	[DECEASED_FORECAST] [INT] NULL,
	[DECEASED_FORECAST_LOW] [INT] NULL,
	[DECEASED_FORECAST_HIGH] [INT] NULL
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_CBS_DECEASED_PER_WEEK_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[CBS_DECEASED_PER_WEEK_NL] (
        [YEAR],
        [WEEK],
        [DECEASED_ACTUAL]
    )
    SELECT
        -- WE MAKE SUBSTRINGS OF THE GIVEN PERIODEN COLUMN. THIS WILL WORK UNTIL THE YEAR
        -- 9999 WHICH SEEMS ADEQUATE FOR NOW
        CAST(SUBSTRING([PERIODS], 1, 4) AS INT),
        CAST(SUBSTRING([PERIODS], 7, 2) AS INT),
        [DECEASED_1]
    FROM
        [VWSSTAGE].[CBS_DECEASED_PER_WEEK_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_DECEASED_PER_WEEK_NL])
END;
GO

### **<span style='color:cadetblue'>FORECAST</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_CBS_DECEASED_PER_WEEK_FORECAST_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[CBS_DECEASED_PER_WEEK_FORECAST_NL] (
        [YEAR],
        [WEEK],
        [DECEASED_FORECAST],
        [DECEASED_FORECAST_LOW],
        [DECEASED_FORECAST_HIGH]
    )
    SELECT
        CAST(SUBSTRING([PERIODS], 1, 4) AS INT),
        CAST(SUBSTRING([PERIODS], 7, 2) AS INT),
        [EXPECTED_AMOUNT_DECEASED],
        [LOWER_LIMIT],
        [UPPER_LIMIT]
    FROM
        [VWSSTAGE].[CBS_DECEASED_PER_WEEK_FORECAST_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_DECEASED_PER_WEEK_FORECAST_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>DECEASED PER WEEK NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[CBS_DECEASED_PER_WEEK_NL]'))
CREATE TABLE [VWSDEST].[CBS_DECEASED_PER_WEEK_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[WEEK_START] [DATETIME] NULL,
	[WEEK_END] [DATETIME] NULL,
	[WEEK_START_UNIX] [BIGINT] NULL,
	[WEEK_END_UNIX] [BIGINT] NULL,
	[YEAR] [INT] NULL,
	[WEEK] [INT] NULL,
	[DECEASED_ACTUAL] [INT] NULL,
	[DECEASED_FORECAST] [INT] NULL,
	[DECEASED_FORECAST_HIGH] [INT] NULL,
	[DECEASED_FORECAST_LOW] [INT] NULL
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_WK_CBS_DECEASED_PER_WEEK_NL] 
	ON [VWSDEST].[CBS_DECEASED_PER_WEEK_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_WK_CBS_DECEASED_PER_WEEK_NL]
    ON [VWSDEST].[CBS_DECEASED_PER_WEEK_NL] (
		[DATE_LAST_INSERTED],
		[WEEK_START]
	)
	INCLUDE (
		[WEEK_START_UNIX], 
		[WEEK_END_UNIX], 
		[DECEASED_ACTUAL], 
		[DECEASED_FORECAST], 
		[DECEASED_FORECAST_HIGH], 
		[DECEASED_FORECAST_LOW]
	);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_CBS_DECEASED_PER_WEEK_NL]
AS
BEGIN
    WITH CTE AS (
        SELECT
            *
        FROM
            [VWSINTER].[CBS_DECEASED_PER_WEEK_NL]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[CBS_DECEASED_PER_WEEK_NL])        
    ),
    DATES_CTE AS(
        SELECT
            [dbo].[CONVERT_ISO_WEEK_TO_DATETIME](T1.[YEAR], T1.[WEEK]) AS [WEEK_START],
            [dbo].[WEEK_END]([DBO].[CONVERT_ISO_WEEK_TO_DATETIME](T1.[YEAR], T1.[WEEK])) AS [WEEK_END],
            [dbo].[CONVERT_WEEKNUMBER_TO_UNIX](T1.[YEAR], T1.[WEEK]) AS [WEEK_START_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DBO].[WEEK_END]([DBO].[CONVERT_ISO_WEEK_TO_DATETIME](T1.[YEAR], T1.[WEEK]))) AS [WEEK_END_UNIX],
            CASE WHEN T1.[WEEK] = 0 THEN T1.[YEAR] - 1 ELSE T1.[YEAR] END AS [YEAR],
            CASE WHEN T1.[WEEK] = 0 THEN 53 ELSE T1.[WEEK] END AS [WEEK],
            T1.[DECEASED_ACTUAL]
        FROM 
            CTE AS T1
    ),
    DECEASED_CTE AS (
        SELECT 
            [WEEK_START], 
            [WEEK_END], 
            [WEEK_START_UNIX], 
            [WEEK_END_UNIX], 
            [YEAR], 
            [WEEK], 
            SUM([DECEASED_ACTUAL]) AS [DECEASED_ACTUAL]
        FROM 
            DATES_CTE
        GROUP BY [WEEK_START], [WEEK_END], [WEEK_START_UNIX], [WEEK_END_UNIX], [YEAR], [WEEK]
    )
    INSERT INTO [VWSDEST].[CBS_DECEASED_PER_WEEK_NL] (
        [WEEK_START],
        [WEEK_END],
        [WEEK_START_UNIX],
        [WEEK_END_UNIX],
        [YEAR],
        [WEEK],
        [DECEASED_ACTUAL],
        [DECEASED_FORECAST],
        [DECEASED_FORECAST_HIGH],
        [DECEASED_FORECAST_LOW]
    )
    SELECT
        T1.[WEEK_START],
        T1.[WEEK_END],
        T1.[WEEK_START_UNIX],
        T1.[WEEK_END_UNIX],
        T1.[YEAR],
        T1.[WEEK],
        T1.[DECEASED_ACTUAL],
        T2.[DECEASED_FORECAST],
        T2.[DECEASED_FORECAST_HIGH],
        T2.[DECEASED_FORECAST_LOW]
    FROM 
        DECEASED_CTE AS T1
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSINTER].[CBS_DECEASED_PER_WEEK_FORECAST_NL]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[CBS_DECEASED_PER_WEEK_FORECAST])
    ) AS T2
    ON T1.[YEAR]=T2.[YEAR] AND T1.[WEEK]=T2.[WEEK]
END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_CBS_DECEASED_PER_WEEK_NL] AS
    SELECT
        [DECEASED_ACTUAL] AS [REGISTERED],
        [DECEASED_FORECAST] AS [EXPECTED],
        [DECEASED_FORECAST_LOW] AS [EXPECTED_MIN],
        [DECEASED_FORECAST_HIGH] AS [EXPECTED_MAX],
        [WEEK_START_UNIX] AS [DATE_START_UNIX],
        [WEEK_END_UNIX] AS [DATE_END_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
    FROM 
        [VWSDEST].[CBS_DECEASED_PER_WEEK_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[CBS_DECEASED_PER_WEEK_NL] WITH(NOLOCK))
        AND [WEEK_START] >= '2020-03-16 00:00:00.000'
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_CBS_DECEASED_PER_WEEK_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'deceased_cbs',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO